## Chapter19: Concurrency Models in Python

#### Concurrency vs. paralellism

concurrency is about dealing with so many things at once but not paralell. paralellism means doing them all at the same time. In concurrency we usually give the system the chance to do many things by improving each in a short time.

The main problem with concurrent programming is that you don't know where exactly the running process goes. When you have a function, you know when it ends and how to get the output but in a Thread, you need something like a message queue to handle the thread execution.

coroutines are simpler than threads to start and get a value from; but they are hard to monitor like threads.


#### common words

|              **Term**                |           **Definition**              |
|----------------------------------|------------------------------------|
|        Concurrency               | The ability to handle multiple pending tasks, making progress one at a time or in parallel (if possible) so that each of them eventually succeeds or fails.|
|         Paralellism              | The ability to execute multiple computations at **the same time**.|
|Execution Unit | Objects that execute code concurrently. Three main kinds: `threads, processes, coroutine`|


#### Execution Units

| **Term**     | **Definition** |
| ------------ | ----------- |
| **Process**  | An independent running program with its own memory space. Processes can have child processes and communicate via serialization (pipes, sockets). Uses **preemptive multitasking** managed by OS. |
| **Thread**   | An execution unit within a process. Threads share the same memory space, allowing easy data sharing but risking data corruption if not synchronized. Also uses **preemptive multitasking**. |
| **Coroutine** | A function that can pause (yield/await) and resume later. Runs cooperatively within an event loop, enabling **asynchronous programming** with minimal resource use compared to threads or processes. |
| **Queue**    | A data structure (usually FIFO) for passing items between execution units safely. Different implementations exist for threads (`queue`), multiprocessing, and async (`asyncio`). |
| **Lock**     | An object to synchronize access to shared data, preventing data corruption by allowing only one execution unit at a time to modify it. Also called a mutex (mutual exclusion). |
| **Contention** | Competition for limited resources, e.g., multiple threads/processes wanting the CPU (CPU contention) or a shared lock (resource contention). Leads to waiting and performance impact. |

#### GIL(General Interpreter Lock)

GIL is the a lock  that is used in cpython to ensure that **only one thread executes Python bytecode at a time**, even if multiple threads exist. It actually protects tha internal data structures and ensures that the memory management is thread-safe.



#### Spinner Example with threads

In [1]:
import itertools
import time
from threading import Thread, Event


def spin(msg: str, done:Event):
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

In [5]:
def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('Thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main():
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == "__main__":
    main()

spinner object: <Thread(Thread-192 (spin), initial)>
Answer: 42  


#### Spinner Example with process

In [6]:
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize

def spin(msg: str, done: synchronize.Event):
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def slow() -> int:
    time.sleep(3)
    return 42

In [10]:
def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('Thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result


def main():
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == "__main__":
    main()

spinner object: <Process name='Process-3' parent=31951 initial>
Answer: 42  


#### Spinner Example with coroutines

!WARNING: You can not run event loops of coroutines in jupyter notebook. so copy the code in simple python file.

In [4]:
import asyncio
import itertools

async def spin(msg: str) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, flush=True, end='')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

async def slow() -> int:
    await asyncio.sleep(3)
    return 42

In [ ]:
import asyncio

async def supervisor() -> int:
    spinner = asyncio.create_task(spin('Thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()
    return result 

def main() -> None:
    result = asyncio.run(supervisor())
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

### An Experiment

in this code you won't see the spinner. why? beacause the app that has a coroutine only has one line of execution and when you run multiple coroutines, it just passes the control between coroutines.

**Concurrency is achieved by control passing from one coroutine to another.**


**IMPORTANT:** Never use `time.sleep()` in a coroutine. it blocks the whole main thread. if a coroutine needs to spend some time doing nothing use `asyncio.sleep` instead.

In [ ]:
import asyncio
import itertools
import time

async def spin(msg: str) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, flush=True, end='')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

async def slow() -> int:
    time.sleep(5)
    return 42

async def supervisor() -> int:
    spinner = asyncio.create_task(spin('Thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()
    return result 

def main() -> None:
    result = asyncio.run(supervisor())
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

## Coroutines vs. Threads

|     Threads     |    Coroutines    |
|-----------------|------------------|
|you must remember to hold locks to protect the critical sections. | your code is protected against intrruption by default.|


## GIL



In [5]:
import math

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    root = math.isqrt(n)
    for i in range (3, root + 1, 2):
        if n % i == 0:
            return False
    return True

In [7]:
%timeit is_prime(5000111000222021)

582 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now first answer these questions without implementation. Then, go on and implement to see How GIL works.

What would happen to the spinner animation if you made the following changes, assuming that n = 5_000_111_000_222_021—that prime which my machine takes 3.3s to verify:

1. In `spinner_proc.py`, replace `time.sleep(3)` with a call to `is_prime(n)`?
2. In `spinner_thread.py`, replace time.sleep(3) with a call to `is_prime(n)`?
3. In `spinner_async.py`, replace await `is_prime(n)`?


##### Conclusion

**GIL**: The GIL is a mutex that allows only one thread to execute Python bytecode at a time in a CPython interpreter process.

GIL affects the concurrency types differently:

- **Processes**: each process has its own interpreter and its own GIL. --> Ture parallelism.

- **Threads**: Threads share one process and one GIL. --> Context switch

- **Coroutines**: Run in single thread and process. --> Cooperative Multitasking


In [10]:
import math

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    root = math.isqrt(n)
    for i in range (3, root + 1, 2):
        if n % i == 0:
            return False
    return True

NUMBERS = [2, 142702110479723, 299593572317531, 3333333333333301, 3333333333333333, 3333335652092209, 4444444444444423, 4444444444444444, 4444444488888889, 5555553133149889, 5555555555555503
, 5555555555555555, 6666666666666666, 6666666666666719, 6666667141414921, 7777777536340681, 7777777777777753, 7777777777777777, 9999999999999917, 9999999999999999]

In [ ]:
from time import perf_counter
from typing import NamedTuple


class Result(NamedTuple):
    prime: bool
    elapsed: float

def check(n: int) -> Result:
    t0 = perf_counter()
    prime = is_prime(n)
    return Result(prime, perf_counter() - t0)

def main() -> None:
    print(f'Checking {len(NUMBERS)} numbers sequentially:')
    t0 = perf_counter()
    for n in NUMBERS:
        prime, elapsed = check(n)
        label = 'P' if prime else " "
        print(f'{label:3} {n:6} {elapsed:9.6f}s')

    elapsed = perf_counter() - t0
    print(f'Total time: {elapsed:.2f}s')

if __name__ == '__main__':
    main()

Checking 20 numbers sequentially:
P        2  0.000000s
P   142702110479723  0.108422s
P   299593572317531  0.142095s
P   3333333333333301  0.478041s
    3333333333333333  0.000003s
    3333335652092209  0.480353s
P   4444444444444423  0.563003s
    4444444444444444  0.000000s
    4444444488888889  0.546856s
    5555553133149889  0.610841s
P   5555555555555503  0.614783s
    5555555555555555  0.000003s
    6666666666666666  0.000000s
P   6666666666666719  0.675547s
    6666667141414921  0.670985s
    7777777536340681  0.725642s
P   7777777777777753  0.723081s
    7777777777777777  0.000003s
P   9999999999999917  0.873790s
    9999999999999999  0.000003s
Total time: 7.21s


In [20]:
## procs.py
import sys
from time import perf_counter
from typing import NamedTuple
from multiprocessing import Process, SimpleQueue, cpu_count, queues

class PrimeResult(NamedTuple):
    n: int
    prime: bool
    elapsed: float

JobQueue = queues.SimpleQueue[int]
ResultQueue = queues.SimpleQueue[PrimeResult]

def check(n: int) -> None:
    t0 = perf_counter()
    res = is_prime(n)
    return PrimeResult(n, res, perf_counter()-t0)

def worker(jobs: JobQueue, results: ResultQueue) -> None:
    while n := jobs.get():
        results.put(check(n))
    results.put(PrimeResult(0, False, 0.0))

def start_jobs(
    procs: int, jobs: JobQueue, results: ResultQueue
) -> None:
    for n in NUMBERS:
        jobs.put(n)
    for _ in range(procs):
        proc = Process(target=worker, args=(jobs, results))
        proc.start()
        jobs.put(0)

def main():
    procs = cpu_count()

    print(f'Checking {len(NUMBERS)} numbers with {procs} processes:')
    t0 = perf_counter()
    jobs: JobQueue = SimpleQueue()
    results: ResultQueue = SimpleQueue()
    start_jobs(procs, jobs, results)
    checked = report(procs, results)
    elapsed = perf_counter() - t0
    print(f'{checked} checks in {elapsed:.2f}s')


def report(procs: int, results:ResultQueue) -> int:
    checked = 0
    procs_done = 0
    while procs_done < procs:
        n, prime, elapsed = results.get()
        if n == 0:
            procs_done += 1
        else:
            checked += 1
            label = 'P' if prime else ' '
            print(f'{n:16}   {label}   {elapsed:9.6f}s')
    return checked

if __name__=='__main__':
    main()

Checking 20 numbers with 24 processes:
               2   P    0.000008s
3333333333333333        0.000017s
4444444444444444        0.000003s
5555555555555555        0.000008s
6666666666666666        0.000000s
7777777777777777        0.000010s
9999999999999999        0.000009s
 142702110479723   P    0.115831s
 299593572317531   P    0.171457s
3333333333333301   P    0.535311s
3333335652092209        0.553660s
4444444444444423   P    0.633849s
4444444488888889        0.630774s
5555553133149889        0.822176s
5555555555555503   P    0.830275s
6666666666666719   P    0.832808s
6666667141414921        0.848129s
7777777777777753   P    0.905185s
7777777536340681        0.972274s
9999999999999917   P    1.005811s
20 checks in 1.04s


In [ ]:
## The number of cores for CPU calculation
cpu_count()

24

### Chapter Summary

-  CPU-intensive fucntions must be avoided in `asyncio`, as they block the event loop.
-  Only processes allow python to benefit from multi core CPUs. 
-  Pythons's GIL makes threads worse than sequential code for heavy computations.
